In [ ]:
import sys
sys.path.append('/home/SharedFolder/git/tillvolkmann/night-drive')
sys.path.append('/home/SharedFolder/git/tillvolkmann/night-drive/classifier')
sys.path.append('/home/SharedFolder/git/tillvolkmann/night-drive/utils')

In [ ]:
import os
import cv2
import time
import glob
import shutil
import random
import subprocess
import numpy as np
#from eval_utils import *

In [ ]:
# specify device
device = "cuda"

# specify paths
path_to_videos = "/home/SharedFolder/CurrentDatasets/bdd100k_video_samples"
weights_classification = "/home/SharedFolder/trained_models/night-drive/timeofday_classifier/resnet18_timeofday_daynight_classifier_best.pth"

# confidence thresholds, set between 0 and 1 to enable module
conf_thresh_classification = 0.7

# mapping dict for timeofday predictions
dict_timeofday = {
     0: "Night",
     1: "Day"
}

# for separating temporary folders when using multiple workers
worker_name = "worker_CAM2"

In [ ]:
outfile_suffix = "_timeofday"
print(outfile_suffix)

In [ ]:
videos = glob.glob(path_to_videos + "/*.mov")
random.seed(123)
random.shuffle(videos)

In [ ]:
def get_CAM_resnet18_BC(image_path, weights_path, class_dict, n_outputs, device):
    """
    CAM code from https://github.com/metalbubble/CAM/blob/master/pytorch_CAM.py
    """
    import cv2
    import torch
    import torch.nn as nn
    from PIL import Image
    from torch.nn import functional as F
    from torchvision import models, transforms
    from torch.autograd import Variable

    net = models.resnet18(pretrained=True)
    finalconv_name = 'layer4'
    net.fc = nn.Linear(net.fc.in_features, n_outputs)
    net.load_state_dict(torch.load(weights_path)["model_state_dict"])
    net.eval()

    # hook the feature extractor
    features_blobs = []

    def hook_feature(module, input, output):
        features_blobs.append(output.data.cpu().numpy())

    net._modules.get(finalconv_name).register_forward_hook(hook_feature)

    # get the softmax weight
    params = list(net.parameters())
    weight_softmax = np.squeeze(params[-2].data.numpy())

    def returnCAM(feature_conv, weight_softmax):
        # generate the class activation maps upsample to 256x256
        size_upsample = (256, 256)
        bz, nc, h, w = feature_conv.shape
        output_cam = []
        cam = weight_softmax.dot(feature_conv.reshape((nc, h * w)))
        cam = cam.reshape(h, w)
        cam = cam - np.min(cam)
        cam_img = cam / np.max(cam)
        cam_img = np.uint8(255 * cam_img)
        output_cam.append(cv2.resize(cam_img, size_upsample))
        return output_cam

    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    preprocess = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor(), normalize])

    img_pil = Image.open(image_path)
    img_tensor = preprocess(img_pil)
    img_variable = Variable(img_tensor.unsqueeze(0))

    net.to(torch.device(device))
       
    img_variable = img_variable.to(torch.device(device))
    logit = net(img_variable)
    logit = logit.cpu()
    
    #h_x = F.softmax(logit, dim=1).data.squeeze()
    #probs, idx = h_x.sort(0, True)
    
    h_x = F.sigmoid(logit).data.squeeze()
    probs, idx = h_x.sort(0, True)
    probs = probs.numpy()
    idx = idx.numpy()
    if (probs) >= 0.5:
        idx = 1
    else:
        idx = 0
    probs = np.append(probs, 1 - probs)
    idx = np.append(idx, 1 - idx)
    sort_idx = np.argsort(probs)
    probs = probs[sort_idx]
    idx = idx[sort_idx]
    probs = probs[::-1]
    idx = idx[::-1]   

    # generate class activation mapping for the top1 prediction
    CAMs = returnCAM(features_blobs[0], weight_softmax)

    # render the CAM and output
    img = cv2.imread(image_path)
    height, width, _ = img.shape
    heatmap = cv2.applyColorMap(cv2.resize(CAMs[0], (width, height)), cv2.COLORMAP_JET)
    result_bgr = heatmap * 0.3 + img * 0.5

    return class_dict[idx[0]], probs[0], result_bgr

In [ ]:
def process_video(video, dict_timeofday, conf_thresh_classification, device):
    # (re-) create directories for extracted frames and target video
    file_name = os.path.basename(video)
    path_name = os.path.dirname(video)
    temp_path = os.path.join(path_name, worker_name, file_name.split(".")[0])
    temp_pred_path = os.path.join(temp_path, "prediction")
    target_path = os.path.join(path_name, "demovideos_CAM")
    target_file = os.path.join(target_path,file_name.split(".mov")[0] + outfile_suffix + ".mp4")
    if not os.path.isdir(target_path):
        os.makedirs(target_path, exist_ok = True)
    elif os.path.exists(target_file):
        # do nothing if file already processed
        return 0    
    if os.path.isdir(temp_path):
        shutil.rmtree(temp_path)
    os.makedirs(temp_path, exist_ok = True)
    os.makedirs(temp_pred_path, exist_ok = True)
    # extract frames from video
    bash_cmd = ["ffmpeg", "-i", video, "-start_number", "0", "-qscale:v", "2", temp_path + "/frame-%d.jpg"]
    subprocess.call(bash_cmd)
    # process frames
    frames = glob.glob(temp_path + "/*.jpg")
    for frame in frames:
        # classify weather and get CAM
        pred_weather_class, pred_weather_score, cam_bgr = get_CAM_resnet18_BC(frame, weights_classification, dict_timeofday, 1, device)
        # write weather on image
        weather_color = [(255, 255, 255) if pred_weather_score >= conf_thresh_classification else (80, 80, 80)]
        cv2.putText(cam_bgr,
                    f"{pred_weather_class} ({pred_weather_score:.2f})", 
                    (5, 715), 
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    0.8, 
                    weather_color[0], 
                    2)
        # write bgr (will be transformed to rgb by open cv)
        cv2.imwrite(os.path.join(temp_pred_path, os.path.basename(frame)), 
                    cam_bgr, 
                    [int(cv2.IMWRITE_JPEG_QUALITY), 95])
    # construct video
    bash_cmd = ["ffmpeg", "-r", "30", "-f", "image2", "-i", temp_pred_path + "/frame-%d.jpg", "-vcodec", "libx264", "-crf", "18", target_file]
    subprocess.call(bash_cmd)
    # clean-up
    shutil.rmtree(temp_path)
    return len(frames)

In [ ]:
for i in range(len(videos)):
    print(f"Processing video {i + 1} of {len(videos)}", end = "")
    tic = time.time()
    n_frames = process_video(videos[i], dict_timeofday, conf_thresh_classification, device)
    toc = time.time()
    if n_frames > 0:
        print(f"... done in {toc - tic:.2f}s ({((toc - tic) / n_frames):.2f}s / frame)")
    else:
        print(f"... skipped. File exists.")